In [ ]:
import pandas as pd
import numpy as np
import math as math

In [ ]:
df = pd.read_stata("usa_00007.dta")

df

In [ ]:
#Data Quality filtering 
df = df[df['qage']!='allocated']
df = df[df['qsex']!='allocated']
df = df[df['qrace']!='allocated']
df = df[df['qwkswork2']!='allocated']
df = df[df['qincwage']!='allocated']
df = df.drop(columns=['qage','qsex','qrace','qwkswork2','qincwage'])
df

In [ ]:
#Complexity of the data has meant seperating hispanic from race to be technically challenging due to crossovers
#eg. (white and hispanic) 
df = df.drop(columns=["raced"])
df

In [ ]:
#Sub-sampling for black and white people only
df = df[(df['race']=='white')|(df['race']=='black/african american/negro')]
df['black'] = df['race'].apply(lambda x: 1 if (x=='black/african american/negro') else 0)
df = df.drop(columns=['race'])

In [ ]:
#Cleaning up the education variable
df["education"] = df["us2010a_schl"] + df["us2019a_schl"]
df = df.drop(columns = ["us2010a_schl","us2019a_schl"])
df['education']=df['education'].astype(str).astype(int)

In [ ]:
#Convert age to integer
df['age']=df['age'].astype(str).astype(int)

In [ ]:
#Convert year to integer
df['year']=df['year'].astype(str).astype(int)

In [ ]:
df

In [ ]:
#Create a male dummy
df['male']=df['sex'].apply(lambda x: 1 if (x=="male") else 0)
df = df.drop(columns=['sex'])
df['male'].astype(str).astype(int)

In [ ]:
pce = pd.read_csv("pce.csv")
pceave = pd.DataFrame(pce.mean())
pceave = pceave.T
deflator2010 = float(pceave['1989']/pceave['2010'])
deflator2019 = float(pceave['1989']/pceave['2019'])

$\text{income in 2010/19}*\frac{PCE:\;1989}{PCE:\;2010/19}=\text{2010/19 income in 1989 prices}$

In [ ]:
#Formatting number of weeks worked
df = df[df['wkswork2']!="n/a"]
df['wkswork2'] = df['wkswork2'].astype(str) 

#Transforming year wages into log weekly earnings
conditions = [
    (df['wkswork2'] == '1-13 weeks'),
    (df['wkswork2'] == '14-26 weeks'),
    (df['wkswork2'] == '27-39 weeks'),
    (df['wkswork2'] == '40-47 weeks'),
    (df['wkswork2'] == '48-49 weeks'),
    (df['wkswork2'] == '50-52 weeks')
    ]
choices = [7, 20, 33, 44, 48, 52]
#52 weeks because 50-51 is probably less likely
df['midpoint'] = np.select(conditions, choices)
df = df.drop(columns=["wkswork2"])

#Converting into real terms
df10=df[df['year']==2010]
df19=df[df['year']==2019]
df10['incwage'] = df10['incwage']*deflator2010
df19['incwage'] = df19['incwage']*deflator2019
df = pd.concat([df10,df19])

#natural log and formatting
df = df[df['incwage']!=0]
df['lwage'] = np.log(df['incwage']/df['midpoint'])
df = df[df['lwage']>0]
df = df.drop(columns=['midpoint','incwage'])

In [ ]:
#Formatting education variables
df = df[df['education']>7]
conditions = [
    (df['education'] == 8) | (df['education'] == 9) | (df['education'] == 10) | (df['education'] == 11),
    (df['education'] == 12),
    (df['education'] == 13),
    (df['education'] == 14) | (df['education'] == 15),
    (df['education'] == 16) | (df['education'] == 17),
    (df['education'] == 18) | (df['education'] == 19),
    (df['education'] == 20), 
    (df['education'] == 21),
    (df['education'] == 22),
    (df['education'] == 23),
    (df['education'] == 24)
    ]
choices = [8,9,10,11,12,13,14,16,18,19,20]
df['educ'] = np.select(conditions, choices)
df = df.drop(columns=['education'])
#Note that in the original used by ACFV educ values can take on 14/15 and 16/17
#these represent different levels of associate and bachelors degrees respectively
#The data does not split these two levels of educations i.e. we have mapped
#14 = associates and 16 = bachelors, and no mapping for 15 and 17.

In [ ]:
#Generate experience variable
df['exper'] = df['age'] - df['educ'] - 6
df['expersq'] = df['exper']**2

In [ ]:
df['perwt'] = df['perwt']/df['perwt'].mean()

In [ ]:
df['male'].dtype

In [ ]:
df['black'].unique()

In [ ]:
df10 = df[df['year']==2010] # obs in 2010 sample
df19 = df[df['year']==2019] # obs in 2019 sample

In [ ]:
df10m = df[(df['year']==2010) & (df['male']==1)]
df10f = df[(df['year']==2010) & (df['male']==0)]
df19m = df[(df['year']==2019) & (df['male']==1)]
df19f = df[(df['year']==2019) & (df['male']==0)]

In [ ]:
df10m = df10m.drop(columns=['male'])
df10f = df10f.drop(columns=['male'])
df19m = df19m.drop(columns=['male'])
df19f = df19f.drop(columns=['male'])

In [ ]:
df10m.to_csv('C:/Users/Nish/OneDrive - University of Bristol/TB2/Econometrics Beyond the Mean/Formative Project/df10m.csv')
df10f.to_csv('C:/Users/Nish/OneDrive - University of Bristol/TB2/Econometrics Beyond the Mean/Formative Project/df10f.csv')
df19m.to_csv('C:/Users/Nish/OneDrive - University of Bristol/TB2/Econometrics Beyond the Mean/Formative Project/df19m.csv')
df19f.to_csv('C:/Users/Nish/OneDrive - University of Bristol/TB2/Econometrics Beyond the Mean/Formative Project/df19f.csv')